In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
file = "../data/raw/amazon_sales_data.csv"
df = pd.read_csv(file,low_memory=False)

# low_memory=False used to avoid dtype inference issues
# Raw dataset contains mixed-type columns (common in e-commerce data)

df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [3]:
df.sample(5)

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
107610,107610,408-0135727-6183517,06-15-22,Shipped,Amazon,Amazon.in,Expedited,J0011,J0011-LCD-S,Set,...,INR,1610.0,BENGALURU,KARNATAKA,560066.0,IN,NaN,True,NaN,False
36229,36229,406-3514336-9093963,04-09-2022,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3739,JNE3739-KR-L,kurta,...,INR,459.0,HYDERABAD,TELANGANA,500049.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
5236,5236,404-8350077-5573121,04-27-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,SET386,SET386-KR-NP-XXL,Set,...,INR,631.0,SURAT,Gujarat,395007.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
108176,108176,405-5384972-2167556,06-15-22,Shipped,Amazon,Amazon.in,Expedited,J0157,J0157-DR-XL,Western Dress,...,INR,908.0,NEW DELHI 110058,DELHI,110058.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False
127895,127895,408-6030518-7156355,06-01-2022,Shipped,Amazon,Amazon.in,Expedited,MEN5015,MEN5015-KR-M,kurta,...,INR,0.0,BENGALURU,KARNATAKA,560043.0,IN,NaN,False,NaN,False


In [4]:
df.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')

- order_id: unique identifier for each order
- date: order transaction date
- fulfilment: fulfillment method (Amazon or Merchant)
- ship_service_level: delivery service type (standard or expedited)
- category: product category of the ordered item
- qty: quantity ordered per line item
- amount: total order value for the line item
- courier_status: shipment progress status
- ship_city, ship_state, ship_country: delivery location details



The most analytically relevant columns are product category, order amount, quantity, shipment service level, order status, and delivery location, as these directly influence demand analysis, revenue contribution, and sales performance.

In [5]:
df = df.drop(['index','Unnamed: 22'],axis=1)

In [6]:
df.columns = df.columns.str.strip().str.replace('[ -]','_',regex =True).str.lower()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   order_id            128975 non-null  object 
 1   date                128975 non-null  object 
 2   status              128975 non-null  object 
 3   fulfilment          128975 non-null  object 
 4   sales_channel       128975 non-null  object 
 5   ship_service_level  128975 non-null  object 
 6   style               128975 non-null  object 
 7   sku                 128975 non-null  object 
 8   category            128975 non-null  object 
 9   size                128975 non-null  object 
 10  asin                128975 non-null  object 
 11  courier_status      122103 non-null  object 
 12  qty                 128975 non-null  int64  
 13  currency            121180 non-null  object 
 14  amount              121180 non-null  float64
 15  ship_city           128942 non-nul

In [8]:
df['date'] = pd.to_datetime(df['date'],
                            format='%Y-%m-%d', 
                            errors='coerce')

In [11]:
df['currency'] = df['currency'].fillna('INR')
df['courier_status'] = df['courier_status'].fillna('Unknown')


In [12]:
df['ship_postal_code'] = df['ship_postal_code'].astype(str)


In [13]:
df['amount'].isna().groupby(df['status']).sum()

status
Cancelled                        7566
Pending                             2
Pending - Waiting for Pick Up       0
Shipped                           208
Shipped - Damaged                   0
Shipped - Delivered to Buyer        8
Shipped - Lost in Transit           0
Shipped - Out for Delivery          0
Shipped - Picked Up                 0
Shipped - Rejected by Buyer         0
Shipped - Returned to Seller        3
Shipped - Returning to Seller       0
Shipping                            8
Name: amount, dtype: int64

In [14]:
df.loc[df['status'] == 'Cancelled', 'amount'] =0
drop_condition = (df['amount'].isna() & (df['status'] != 'Cancelled'))
df = df.drop(df[drop_condition].index)
df = df.reset_index(drop =True)

In [15]:
df = df.dropna(
    subset =  ['ship_city','ship_state','ship_postal_code','ship_country']
    )

In [16]:
df = df.drop(columns= ['promotion_ids'])

In [17]:
df['b2b'] = df['b2b'].astype(int)


In [18]:
print(df['status'].value_counts())

status
Shipped                          77580
Shipped - Delivered to Buyer     28754
Cancelled                        18325
Shipped - Returned to Seller      1947
Shipped - Picked Up                973
Pending                            656
Pending - Waiting for Pick Up      281
Shipped - Returning to Seller      145
Shipped - Out for Delivery          35
Shipped - Rejected by Buyer         11
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: count, dtype: int64


In [19]:
df['fulfilled_by'] = df['fulfilled_by'].fillna('Not Applicable')


The missing values are structural. The column is only relevant for Amazon-fulfilled orders, so for Merchant-fulfilled orders it’s not applicable. I retained it to preserve fulfillment-level analysis.

In [20]:
import os

os.makedirs("../data/cleaned", exist_ok=True)

df.to_csv(
    "../data/cleaned/amazon_sales_clean.csv",
    index=False
)


In [21]:
print(df['status'].value_counts())

status
Shipped                          77580
Shipped - Delivered to Buyer     28754
Cancelled                        18325
Shipped - Returned to Seller      1947
Shipped - Picked Up                973
Pending                            656
Pending - Waiting for Pick Up      281
Shipped - Returning to Seller      145
Shipped - Out for Delivery          35
Shipped - Rejected by Buyer         11
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: count, dtype: int64


In [22]:
df_check = pd.read_csv(
    "../data/raw/amazon_sales_data.csv",
    low_memory=False
    )
df_check.shape

(128975, 24)

In [23]:
from sqlalchemy import create_engine

# NOTE:
# Database credentials are intentionally hidden for security reasons.
# Replace <username> and <password> with your local MySQL credentials to run this.
# This step demonstrates loading cleaned data into MySQL.

engine = create_engine(
    "mysql+mysqlconnector://<username>:<password>@localhost/amazon_sales_analysis"
)

df.to_sql(
    "amazon_sales",
    engine,
    if_exists="replace",
    index=False,
    chunksize=5000
)



ProgrammingError: (mysql.connector.errors.ProgrammingError) 1045 (28000): Access denied for user '<username>'@'localhost' (using password: YES)
(Background on this error at: https://sqlalche.me/e/20/f405)